<a href="https://colab.research.google.com/github/Alexandresl/Projeto-imersao-ia/blob/main/Projeto_para_Imers%C3%A3o_Intelig%C3%AAncia_Artificial_2%C2%AA_Edi%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto para Imersão Inteligência Artificial 2ª Edição

## Triagem de Reclamações com Google Gemini

Este projeto tem como objetivo auxiliar atendentes que recebem reclamações a partir de um SAC (Serviço de Atendimento ao Consumidor) em uma empresa de distribuição de Energia Elétrica, a ingressagem as reclamações utilizando as tipologias corretas para cada cenário.

A escolha do tema foi devido a complexidade que esta classificação alcança na medida que a quantidade de tipologias aumenta. Em um cenário de atendimento real, podem ocorrer erros de cadastro caso o atendente não identifique corretamente a tipologia. Estes erros comprometem a assertividade das equipes que irão tratar estas reclamações, gerando muitas vezes um atraso desnecessário para a solução da demanda do consumnidor.

## Tecnologias utilizadas no projeto

- Linguagem de Programação Python
- Bibliotecas numpy e pandas do Python
- Google API Key
- Google AI Studio
- Google Colab
- Github

## Recursos

O sistema pode receber entrada de dados em dois cenários:

1. O próprio atendente que está realizando o atendimento pode digitar a demanda do cliente e como retorno terá a tipologia adequada.

2. Para o caso da reclamação ser recebida via texto, exemplo, e-mail, formulário web, aplicativos de mensagem, etc. Pode ser copiada toda a mensagem do cliente e receber a tipologia adequada para ingresso da manifestação.

## Observação

Para fim de testes, foram utilizados 3 tipologias que serão posteriormente ampliadas para o cenário real.

In [1]:
# Instalando a API do Gemini no projetro
pip install -U -q google-generativeai

# Importando as dependências necessárias:
import numpy as np
import pandas as pd
import google.generativeai as genai

# Informando a Google API Key
from google.colab import userdata
api_key = userdata.get('API_KEY')
genai.configure(api_key=api_key)

> Abaixo, é construída a lista de tipologias. Cada tipologia possui um título e o conteúdo, que explica quando esta tipologia deverá ser utilizada e alguns cenários de uso.

In [32]:
# Listagem das tipologias que serão utilizadas

tipologia1 = {
"Título": "Falta de Energia",
"Conteúdo": "Utilizada quando o cliente reporta a interrupção no fornecimento de energia elétrica em sua unidade consumidora."
  "Exemplos:"
"- Apagão total ou parcial na residência."
"- Oscilações frequentes na rede elétrica, causando quedas de energia."
"- Falta de energia prolongada, sem aviso prévio por parte da distribuidora."
"- Retorno da energia com tensão inadequada, causando danos a equipamentos."
}

tipologia2 = {
"Título": "Serviço Atrasado",
"Conteúdo": "Utilizada quando o cliente solicita um serviço à distribuidora e este não é realizado dentro do prazo previsto."
"Exemplos:"
"- Ligação nova de energia elétrica."
"- Aumento de carga instalada."
"- Religação de energia após corte por falta de pagamento."
"- Troca de titularidade da conta."
"- Reparo em poste ou fiação da rede elétrica."
"- Poda de árvores que interferem na fiação elétrica."
}

tipologia3 = {
"Título": "Erro de faturamento",
"Conteúdo": "Utilizada quando o cliente identifica irregularidades em sua fatura de energia elétrica."
"Exemplos:"
"- Valor da fatura muito acima do consumo usual."
"- Cobrança de serviços não solicitados ou não realizados."
"- Erro na leitura do medidor de consumo."
"- Cobrança duplicada de um mesmo período."
"- Multa aplicada indevidamente."
"- Inclusão de impostos com valores incorretos."
}

tipologias = [tipologia1, tipologia2, tipologia3]

In [22]:
# Imprime a lista de tipologias em formato tabela
df = pd.DataFrame(tipologias)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Falta de Energia,Utilizada quando o cliente reporta a interrupç...
1,Serviço Atrasado,Utilizada quando o cliente solicita um serviço...
2,Cobrança Indevida,Utilizada quando o cliente identifica irregula...


In [23]:
model = "models/embedding-001"

In [24]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [25]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Falta de Energia,Utilizada quando o cliente reporta a interrupç...,"[0.00027413908, -0.01609326, -0.06327277, 0.03..."
1,Serviço Atrasado,Utilizada quando o cliente solicita um serviço...,"[0.027932707, 0.008274737, -0.03271068, 0.0401..."
2,Cobrança Indevida,Utilizada quando o cliente identifica irregula...,"[-0.01733446, -0.039353162, -0.049394812, 0.01..."


> Criando a função para realizar a consulta:

In [26]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Titulo"]

> Realizando três consultas utilizando como cenário o caso do próprio atendente estar consultando o sistema para identificar a tipologia correta:




In [27]:
consulta = "Cliente reclama que solicitou ligação e não foi atendido"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Serviço Atrasado


In [29]:
consulta = "Cliente reclama que sua conta está cara."

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Cobrança Indevida


In [30]:
consulta = "Cliente informa que chegou em casa e não tinha luz."

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Falta de Energia


> Esta nova consulta, testa o cenário em que pegamos um texto encaminhado pelo próprio cliente onde a IA interpreta o teor da reclamação e indica em qual tipologia esta manifestação deverá ser classificada.

In [31]:
consulta = "Quero reclamar que minha conta está muito cara. Quase não paro em casa e cada mês que passa pago mais de luz."

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Cobrança Indevida
